<a href="https://colab.research.google.com/github/Muhammad-Yunus/Colab/blob/main/Grounding_DINO_%2B_Autodistill_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Autodistill banner](https://camo.githubusercontent.com/29d0935e1b88b0ebb784c22ef6dad74e998ddb8d7f34fec315efe2279e5d9d1d/68747470733a2f2f6d656469612e726f626f666c6f772e636f6d2f6f70656e2d736f757263652f6175746f64697374696c6c2f6175746f64697374696c6c2d62616e6e65722e6a70673f32)

# Grounding DINO + Autodistill Evaluation

This notebook contains the code used to evaluate Grounding DINO for our Autodistill + Grounding DINO Evaluation blog post.

We will use Grounding DINO to annotate images in five datasets and visualize the results for 16 images in each dataset.

## Configure GPU

First, make sure you are using a GPU in this Notebook. Cclick Edit -> Notebook settings -> Hardware accelerator, set it to GPU, and then click Save.

## Install Dependencies

Next, install the required dependencies by running the code cell below.

In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [19]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [ ]:
!pip install roboflow autodistill autodistill-grounding-dino supervision -q

## Download a Dataset

First, we need a dataset with which to work. Below, paste in a URL to any dataset on Roboflow Universe. You can also download data on your private account by uncommenting the commented code below.

In [ ]:
import roboflow

roboflow.login()

datasets = [
    "https://universe.roboflow.com/mohamed-traore-2ekkp/taco-trash-annotations-in-context/model/16",
    "https://universe.roboflow.com/roboflow-universe-projects/retail-coolers/model/12",
    "https://universe.roboflow.com/roboflow-universe-projects/people-and-ladders/model/4",
    "https://universe.roboflow.com/roboflow-universe-projects/safety-cones-vfrj2/model/4",
    "https://universe.roboflow.com/roboflow-universe-projects/people-detection-thermal/deploy/5"
]

for dataset in datasets:
  roboflow.download_dataset(dataset_url=dataset, model_format="yolov8")

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="79mx7RziLPHPZMmphjFI")
project = rf.workspace("learning-kqfkj").project("skissors-detection")
version = project.version(1)
dataset = version.download("yolov8")


## Import Dependencies

Below, we import the dependencies required for our project.

In [ ]:
from autodistill_grounding_dino import GroundingDINO
from autodistill.detection import CaptionOntology
import supervision as sv
import cv2
import os
import torch
import random

torch.use_deterministic_algorithms(False)

## Label Images with Grounding DINO

Below, we set an "ontology". Ontologies map prompts -- text given to a model for use in labelling data -- to the labels you want your dataset to include.

For example, the ontology `"container": "shipping container"` will send the prompt "container" to the base model (in this example Grounding DINO). All objects matching that prompt will be labelled "shipping container".

In [26]:
DATASET_FOLDER = "./Skissors-Detection-1"
PROJECT_NAME = "Scissors Detection"
ONTOLOGY = {
    "scissors": "scissors"
}

base_model = GroundingDINO(ontology=CaptionOntology(ONTOLOGY))

folder = os.path.join(DATASET_FOLDER, "valid/images/")

image_names = os.listdir(folder)

images = {}
annotations = {}

#all_images = []
#titles = []

for image_name in image_names:

  # find full path of the image /path/to/file.jpg
  image_name = os.path.join(folder, image_name)

  if not image_name.endswith((".jpg", ".jpeg", ".png")):
    print(image_name)
    continue

  predictions = base_model.predict(image_name)

  image = cv2.imread(image_name)

  annotator = sv.BoxAnnotator()

  predictions = predictions[predictions.confidence > 0.5]

  # annotated_image = annotator.annotate(
  #     scene=image,
  #     detections=predictions
  # )

  annotations[image_name] = predictions
  images[image_name] = image
  #all_images.append(annotated_image)
  #titles.append(i.split(".")[0])

#sv.plot_images_grid(all_images, grid_size=(4, 4), titles=titles, size=(24, 24))

trying to load grounding dino directly
final text_encoder_type: bert-base-uncased


In [35]:
detections = annotations['./Skissors-Detection-1/valid/images/IMG_20240914_081441_jpg.rf.34c9b349af988aa081cc3d71466300d0.jpg']

In [38]:
detections.confidence

array([0.9611151], dtype=float32)

In [39]:
detections.class_id

array([0])

In [22]:
ANNOTATIONS_DIRECTORY = os.path.join(HOME, 'annotations')

MIN_IMAGE_AREA_PERCENTAGE = 0.002
MAX_IMAGE_AREA_PERCENTAGE = 0.80
APPROXIMATION_PERCENTAGE = 0.75

CLASSES = ["scissors"]

In [41]:
!pip uninstall -y supervision
!pip install -q supervision==0.6.0

import supervision as sv
print(sv.__version__)

Found existing installation: supervision 0.6.0
Uninstalling supervision-0.6.0:
  Successfully uninstalled supervision-0.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rf-groundingdino 0.1.2 requires supervision>=0.8.0, but you have supervision 0.6.0 which is incompatible.
0.23.0


In [40]:
sv.Dataset(
    classes=CLASSES,
    images=images,
    annotations=annotations
).as_pascal_voc(
    annotations_directory_path=ANNOTATIONS_DIRECTORY,
    min_image_area_percentage=MIN_IMAGE_AREA_PERCENTAGE,
    max_image_area_percentage=MAX_IMAGE_AREA_PERCENTAGE,
    approximation_percentage=APPROXIMATION_PERCENTAGE
)

AttributeError: module 'supervision' has no attribute 'Dataset'